# TP5 : Reconnaissance d'image par un réseau de neurones

### Chargement des librairies

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

### Chargement des données

In [3]:
# Lecture des données
DIAGONAL=pd.read_csv("deuxdiag.csv")
# Visualisation des 5 premières données
DIAGONAL.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y
0,0,0,1,1,0,0,1,0,1,0
1,0,1,1,0,1,0,1,1,0,1
2,1,1,0,0,1,1,1,0,1,1
3,0,0,0,1,1,1,0,0,1,0
4,1,0,1,1,1,0,1,0,0,1


**Exercice**
1. Dans le jeu de données `DIAGONAL`, à quoi correspond une donnée ?
2. On considère les données `X1`, ..., `X9` comme les 9 cases d'un carré 3*3 :<br/>
    X1 X2 X3<br/>
    X4 X5 X6<br/>
    X7 X8 X9<br/>
    À quelle condition sur les `Xi` a-t-on : une des deux diagonales du carré est constituée de 1 ?
3. Vérifier sur les 5 premières données que le jeux de données `DIAGONAL` est tel que la conditon précédente est vérifiée si et seulement si `Y=1`.

### Données d'apprentissage et données de validation

Le jeu de données est séparé 2 :
- Deux tiers des données sont utilisées pour l'apprentissage du réseau de neurones
- Le troisième tiers des données est mis à l'écart et sera utilisé pour évaluer la performance du réseau sur ces données (sur lesquelles il n'aura pas appris).

In [4]:
N=len(DIAGONAL) # Nombre de données
n=int(2*N/3) # Deux tiers du nombres de données 
X=DIAGONAL.drop(['Y'],axis=1)[0:n] # Données d'apprentissage X
Y=DIAGONAL['Y'][0:n] # Données d'apprentissage Y
X.insert(loc = 0, column = 'X0',value = np.ones(n,dtype = int)) # On ajoute une colonne de 1 pour b
Xvalid=DIAGONAL.drop(['Y'],axis=1)[n+1:N] # Données de validation X
Yvalid=DIAGONAL['Y'][n+1:N] # Données de validation Y
nvalid=len(Xvalid)
Xvalid.insert(loc = 0, column = 'X0',value = np.ones(nvalid,dtype=int)) # On ajoute une colonne de 1 pour b

**Exercice**

Sachant que `DIAGONAL` contient 150 données $(X_i,Y_i)$, sur combien de données le réseau va-t-il apprendre ? Sur combien de données la performance du réseau va-t-elle être évaluée ?

Que contiennent les variables `X` et `Y` ?

### Réseau à 1 couche cachée de 2 neurones
On approxime $X\mapsto Y$ par la fonction $R_{\mathcal{W}}$, où $\mathcal{W}$ désigne l'ensemble des paramètres $b$, $w_1$, $w_2$, $B=\begin{pmatrix}b_1\\b_2\end{pmatrix}$ et $W=\begin{pmatrix}W_1\\W_2\end{pmatrix}=\begin{pmatrix}w_{11}&w_{12}&\ldots&w_{19}\\w_{21}&w_{22}&\ldots&w_{29}\end{pmatrix}$, définie par : $$R_{\mathcal{W}}:X\mapsto \sigma\left(b+\begin{pmatrix}w_1&w_2\end{pmatrix}\cdot H(B+W\cdot X)\right),$$ où
$$\sigma(x)=\frac{1}{1+e^{-x}}$$ est la fonction sigmoïde, et $$H=\begin{pmatrix}h_1(X)\\h_2(X)\end{pmatrix}=\begin{pmatrix}\sigma\left(b_1+\begin{pmatrix}w_{11}&\ldots&w_{19}\end{pmatrix}\cdot X\right)\\\sigma\left(b_2+\begin{pmatrix}w_{21}&\ldots&w_{29}\end{pmatrix}\cdot X\right)\end{pmatrix}$$ est la couche cachée.

**Exercice**

1. Combien de paramètres la fonction $R_{\mathcal{W}}$ possède-t-elle ?
2. Représenter le réseau de neurones associé à $R_{\mathcal{W}}$.
3. Pourquoi parle-t-on d'un réseau à *deux* neurones ?

### Rétropropagation du gradient

Les paramètres $\mathcal{W}$ sont déterminés en minimisant la fonction :
$$f(\mathcal{W})=\frac{1}{2}\sum\limits_{j} (R_{\mathcal{W}}(X_j)-Y_j)^2$$
où la somme est calculée sur les données d'apprentissage.

On réécrit $$f(\mathcal{W})=\frac{1}{2}\sum\limits_{j} (\sigma(s_j)-Y_j)^2$$
où $$s_j=b+\begin{pmatrix}w_1&w_2\end{pmatrix}\cdot H_j,$$
avec $$H_j=\begin{pmatrix}\sigma(h_{1j})\\ \sigma(h_{2j})\end{pmatrix}$$
où $$h_{ij}=b_i+W_i\cdot X_j.$$

**Exercice**

On rappelle la propriété $\sigma'(x)=\sigma(x)\times (1-\sigma(x))$.

1. À partir de $$f(\mathcal{W})=\frac{1}{2}\sum\limits_{j} (\sigma(\underbrace{b+\begin{pmatrix}w_1&w_2\end{pmatrix}\cdot H_j}_{s_j})-Y_j)^2$$ établir les dérivées suivantes :
    - $\frac{\partial f(\mathcal{W})}{\partial b}=\sum\limits_{j} (\sigma(s_j)-Y_j)\sigma(s_j)(1-\sigma(s_j))\times1$
    - $\frac{\partial f(\mathcal{W})}{\partial w_i}=\sum\limits_{j} (\sigma(s_j)-Y_j)\sigma(s_j)(1-\sigma(s_j))\sigma(h_{ij})$

2. À partir de $$f(\mathcal{W})=\frac{1}{2}\sum\limits_{j} (\sigma(\underbrace{b+w_1\sigma(\underbrace{b_1+\sum\limits_{k=1..9} w_{1k}x_{kj}}_{h_{1j}})+w_2\sigma(\underbrace{b_2+\sum\limits_{k=1..9} w_{2k}x_{kj}}_{h_{2j}})}_{s_j})-Y_j)^2$$ établir les dérivées suivantes :
    - $\frac{\partial f(\mathcal{W})}{\partial b_i}=\sum\limits_{j} (\sigma(s_j)-Y_j)\sigma(s_j)(1-\sigma(s_j)w_i\sigma(h_{ij})(1-\sigma(h_{ij}))\times 1$
    - $\frac{\partial f(\mathcal{W})}{\partial w_{ik}}=\sum\limits_{j} (\sigma(s_j)-Y_j)\sigma(s_j)(1-\sigma(s_j))w_i\sigma(h_{ij})(1-\sigma(h_{ij}))x_{kj}$

In [6]:
def f(W):
    H1=[sigma(np.dot(W[1],X.iloc[j])) for j in range(n)] # liste des sigma(h1j)
    H2=[sigma(np.dot(W[2],X.iloc[j])) for j in range(n)] # liste des sigma(h2j)
    S=[sigma(W[0][0]+W[0][1]*H1[j]+W[0][2]*H2[j]) for j in range(n)] # liste des sigma(sj)
    return np.sum([(S[j]-Y[j])**2 for j in range(n)])

def sigma(x):
    return 1/(1+np.exp(-x))

# Calcul du gradient
def gradient(W): #W=[[b,w1,w2],[b1,w11,w12,...,w19],[b2,w21,...,w29]]
    H1=[sigma(np.dot(W[1],X.iloc[j])) for j in range(n)] # liste des sigma(h1j)
    H2=[sigma(np.dot(W[2],X.iloc[j])) for j in range(n)] # liste des sigma(h2j)
    S=[sigma(W[0][0]+W[0][1]*H1[j]+W[0][2]*H2[j]) for j in range(n)] # liste des sigma(sj)
    delta=[(S[j]-Y.iloc[j])*S[j]*(1-S[j]) for j in range(n)] # liste des produits utilisés
    # Dérivées partielles :
    db=np.sum([delta[j] for j in range(n)])
    dw1=np.sum([delta[j]*H1[j] for j in range(n)])
    dw2=np.sum([delta[j]*H2[j] for j in range(n)])
    d1=[delta[j]*W[0][1]*H1[j]*(1-H1[j]) for j in range(n)] # liste de produits très utilisés
    d2=[delta[j]*W[0][2]*H2[j]*(1-H2[j]) for j in range(n)] # liste de produits très utilisés
    db1=np.sum([d1[j] for j in range(n)])
    db2=np.sum([d2[j] for j in range(n)])
    dW1=[0,0,0,0,0,0,0,0,0]
    dW2=[0,0,0,0,0,0,0,0,0]
    for k in range(9):
        dW1[k]=np.sum([d1[j]*X.iloc[j][k] for j in range(n)])
        dW2[k]=np.sum([d2[j]*X.iloc[j][k] for j in range(n)])
    return [[db,dw1,dw2],[db1]+dW1,[db2]+dW2]

***Exercice***

1. Calculer (dans la cellule ci-dessous) le gradient de $f$ pour des paramètres tous nuls : $\mathcal{W}_0=(0,\ldots,0)$.
2. En déduire les coordonnées du point $\mathcal{W}_1$ après une première itération de la descente de gradient pour un pas de $\tau=10$.
3. Calculer le gradient en $\mathcal{W}_1$. Que remarquez vous ? Comment l'expliquer ?
4. Est-ce judicieux de partir de $\mathcal{W}_0$ ?

In [170]:
#Calculs :


In [7]:
# Algorithme de descente
def descente(W,tau=1,tolerance=1e-3,Nbiterations=10):
    for i in range(Nbiterations):
        g = gradient(W)
        normg = g[0][0]**2+g[0][1]**2+g[0][2]**2
        +np.sum([g[1][k]**2 for k in range(10)])
        +np.sum([g[2][k]**2 for k in range(10)])
        print('g:',normg) # On affiche la norme du gradient pour visualiser la descente
        print('f:',f(W))
        if normg<tolerance:
            print('L\'algorithme a convergé.\n Solution atteinte:\n W=',W,'\n Norme du gradient:',normg)
            return W
        W[0][0]=W[0][0]-tau*g[0][0]
        W[0][1]=W[0][1]-tau*g[0][1]
        W[0][2]=W[0][2]-tau*g[0][2]
        for j in range(10):
            W[1][j]=W[1][j]-tau*g[1][j]
            W[2][j]=W[2][j]-tau*g[2][j]
    print('L\'algorithme n\'a pas convergé.\n Solution atteinte:\n W=',W,'\n Norme du gradient:',normg)
    return W

***Exercice***

Identifier la monotonie de l'algorithme sur les 10 premières itérations, à partir de 

`W=[[0,0,1],[0,0,0,1,0,1,1,0,1,0],[0,1,0,0,0,1,1,0,1,0]]`

pour un pas de :
- $\tau=0.1$, avec une tolérance de $10^{-10}$ 
- $\tau=1$, avec une tolérance de $10^{-10}$
- $\tau=2$, avec une tolérance de $10^{-20}$

In [75]:
#Calculs
W=[[0,0,1],[0,0,0,1,0,1,1,0,1,0],[0,1,0,0,0,1,1,0,1,0]]
W=descente(W,1,1e-10)

g: 184.02878287905696
f: 37.051174795318545
g: 1.13608433840083e-11
f: 27.999994869855676
L'algorithme a convergé.
 Solution atteinte:
 W= [[-8.929644821667454, -7.2432946299827305, -6.198951947414842], [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0], [-1.2303373644099542, -0.23033736440995423, -0.30526067724020306, -0.5423819063269736, -0.4731645085867943, 0.4935965252249378, 0.6852701072378768, -0.4072848497373618, 0.4960288540003609, -0.4385947514682201]] 
 Norme du gradient: 1.13608433840083e-11


**Exercice**
    
Calculer, à l'aide de la fonction `prediction` ci-dessous, la sortie du réseau de neurones pour une valeur $X$ possèdant une diagonale de $1$ et pour une valeur de $X$ ne possédant pas une diagonale de $1$.

In [77]:
def prediction(W,X):
    return sigma(W[0][0]+W[0][1]*sigma(np.dot(W[1],X))+W[0][2]*sigma(np.dot(W[2],X)))

In [ ]:
# Entrer le W trouvé par l'algorithme de descente ci-dessus
W= ...

In [ ]:
# entrer un X possédant ou ne possédant pas de diagonale de 1 
# attention à insérer un 1 avant les 9 coordonnées de X, correspondant à l'ajout de b
# dans le produit W.X
print(prediction(W,...)


### Validation du modèle

**Exercice**

À l'aide de la fonction suivante, déterminer le nombre d'erreurs du réseau de neurones paramétré par le `W` de l'exercice précédent, calculé sur les données de validations.

In [79]:
print('Nombre d\'erreurs sur les',len(Xvalid),'données de validation :',
      np.sum([np.abs(round(prediction(W,Xvalid.iloc[i]))-Yvalid.iloc[i])
              for i in range(len(Xvalid)-1)])
     )


Nombre d'erreurs sur les 49 données de validation : 9.0


### Exercice
Refaire le calcul précédent (validation du modèle) en utilisant le vecteur de paramètres :

`W= [[8.151766727885503, -18.884915909234774, 20.58237721518839], [12.461460129290158, 1.0068383415839224, 0.6124931603260179, -5.6633462093648435, 0.31937922293433985, -5.499323198858325, 1.1547050495924607, -5.986379875948584, 0.08814543368440875, 1.8230438209745343], [-18.262720764933253, 6.653699501004856, -0.041561815642582586, 0.7187311299131572, 0.7479057784712368, 7.456783033416534, 0.18723100788706293, 0.9007878917526597, 0.26682037735546293, 6.687905945851743]]`

# Utilisation d'une librairie

In [82]:
from sklearn.neural_network import MLPClassifier

In [63]:
XX=DIAGONAL.drop(['Y'],axis=1)[0:n] # Données d'apprentissage X
YY=DIAGONAL['Y'][0:n] # Données d'apprentissage Y
XXvalid=DIAGONAL.drop(['Y'],axis=1)[n+1:N] # Données de validation X
YYvalid=DIAGONAL['Y'][n+1:N] # Données de validation Y

In [102]:
modele = MLPClassifier(
    activation='logistic',
    solver='sgd', # sgd="stochastic gradient descent", #essayer avec lbfgs, ou adam à la place
    shuffle=True,
    hidden_layer_sizes=(2,),
    max_iter=5000,
    tol=1e-20)

In [ ]:
modele.fit(XX,YY)
print('Score : ',modele.score(XXvalid,YYvalid))

In [ ]:
W=[[modele.intercepts_[1][0]]+[modele.coefs_[1][0][0]]+[modele.coefs_[1][1][0]],\
[modele.intercepts_[0][0]]+[modele.coefs_[0][k][0] for k in range(9)],\
[modele.intercepts_[0][1]]+[modele.coefs_[0][k][1] for k in range(9)]]
print(W)

In [ ]:
print('Nb d\'erreurs :',np.sum(\
             [np.abs(modele.predict([XXvalid.iloc[i]])-YYvalid.iloc[i])\
              for i in range(len(XXvalid)-1)]))